In [415]:
import numpy as np
import pandas as pd
from imutils import paths
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [316]:
dataPaths = list(paths.list_files('.\ipl_csv2'))
dataPaths.pop(len(dataPaths) - 1)
dataPaths.pop(len(dataPaths) - 2)



def retrieve_past_data(team_a, team_b, num):
    req_df = []
    # stadium_avg = {}
    # unique_bowlers = set()
    # unique_batsmen = set()

    for match_file in dataPaths:

        try:
            df = pd.read_csv(match_file, index_col='match_id')
            # temp_a, temp_b = get_unique_players(df)
            # unique_bowlers |= (temp_a) 
            # unique_batsmen |= (temp_b) 
            df = preprocess(df)
            # stadium_avg = get_stadium_avg(stadium_avg, df)
            if (df.iloc[1]['batting_team'] == team_a or df.iloc[1]['batting_team'] == team_b) and (df.iloc[1]['bowling_team'] == team_a or df.iloc[1]['bowling_team'] == team_b):
                if len(req_df) < num or num == -1:
                    req_df.append(df)
        
        except Exception as e:
            print(e)

    return req_df


def preprocess(df):
    pp_df = df[df['ball'] <= 5.6]
    pp_df = pp_df[pp_df['innings'] <= 2]

    pp_df['total_runs'] = pp_df['runs_off_bat'] + pp_df['extras']

    return pp_df

#for stadium avg
def get_stadium_avg(stad_dict, df):
    venue = df.iloc[1]['venue']
    runs1 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 1)]['total_runs'].sum()
    runs2 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 2)]['total_runs'].sum()

    if (runs1 <= 150 and runs1 >= 10):
        stad_dict.setdefault(venue,[]).append(runs1)
    if (runs2 <= 150 and runs2 >= 10):
        stad_dict.setdefault(venue,[]).append(runs2)

    return stad_dict

# def get_unique_players(df):
#     bowlers = set(df['bowler'].unique().tolist())
#     batsmen_striker = set(df['striker'].unique().tolist())
#     batsmen_non_striker = set(df['non_striker'].unique().tolist())
    
#     batsmen = batsmen_striker | (batsmen_non_striker)

#     return bowlers, batsmen


In [317]:
#returns a list of dataframes between the two given teams
req_df = retrieve_past_data("Chennai Super Kings", "Kings XI Punjab", -1)

#format the data for visualization (leave it out for now)
# stad_avg['M.Chinnaswamy Stadium'].extend(stad_avg['M Chinnaswamy Stadium'])
# stad_avg.pop('M Chinnaswamy Stadium')

# stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk'])
# stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk, Chennai'])
# stad_avg.pop('MA Chidambaram Stadium, Chepauk')
# stad_avg.pop('MA Chidambaram Stadium, Chepauk, Chennai')

# stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association IS Bindra Stadium, Mohali'])
# stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association Stadium, Mohali'])
# stad_avg.pop('Punjab Cricket Association IS Bindra Stadium, Mohali')
# stad_avg.pop('Punjab Cricket Association Stadium, Mohali')

# stad_avg['Rajiv Gandhi International Stadium'].extend(stad_avg['Rajiv Gandhi International Stadium, Uppal'])
# stad_avg.pop('Rajiv Gandhi International Stadium, Uppal')

# stad_avg['Arun Jaitley Stadium'].extend(stad_avg['Feroz Shah Kotla'])
# stad_avg.pop('Feroz Shah Kotla')

# stad_avg['Wankhede Stadium'].extend(stad_avg['Wankhede Stadium, Mumbai'])
# stad_avg.pop('Wankhede Stadium, Mumbai')

# print(stad_avg)

C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [294]:

# # Importing library
# import csv
  
# # data to be written row-wise in csv fil
# data = list(u_batsmen)
# data.sort()

# with open('batsmen.csv', 'w') as myfile:
#      wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

#      for d in data:
#         wr.writerow([d])

# print(len(data))

544


In [318]:
# stad_names = list(stad_avg.keys())
# keys = pd.Series(stad_names, dtype="category")
# vals = [float(len(stad_avg[k])) for k in keys.cat.categories]


# plt.figure(figsize=(20, 10))
# sns.barplot(x=keys.cat.codes, y=vals)

In [319]:
# dict(enumerate(keys.cat.categories))

In [395]:
#For cross validation, use the last of the req_df as test data and use the rest for training
test = req_df[len(req_df) - 1]
train_set = req_df[:len(req_df) - 1]

#test_x is the test input similar to actual input
test_batsmen = list(set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['striker'].unique()) | set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['non_striker'].unique()))
test_bowlers = test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['bowler'].unique()
test_x = pd.DataFrame({'venue': [test.iloc[2]['venue']], 'innings': [1], 'batting_team': [test.iloc[2]['batting_team']], 'bowling_team':[test.iloc[2]['bowling_team']], 'batsmen': [test_batsmen], 'bowlers': [test_bowlers]})

#used to find the innings of the training match we are going to use for training
team_a = test.iloc[1]['batting_team']
team_b = test.iloc[1]['bowling_team']

test_x.head()

for df in train_set:
    match_id = df.index[0]

    #innings for which we are going to use for training
    inn = df[(df['batting_team'] == team_a) & (df['bowling_team'] == team_b)]['innings'].iloc[0] 

    runs_x = df.loc[(df['ball'] < 6.1) & (df['innings'] == inn)]['total_runs'].sum() #runs of training innings
    runs_target = df.loc[(df['ball'] < 6.1) & (df['innings'] == 3 - inn)]['total_runs'].sum() #runs of other innings
    
    #MODEL CODE GOES HERE
    

    #convert the df into usable training data with required features
    # train_x = pd.DataFrame('venue': [get_venue_score(df)], 'batsmen_rating': [get_batsmen_scores(df)], 'bowlers_rating': [get_bowlers_scores(df)], 'wickets': [get_wickets(df)])

In [ ]:
# def get_venue_score(df):
#     venue = df.iloc[1]['venue']
#     #assign venue scores here

#     return random.uniform(0, 1)

# def get_batsmen_scores(df_list):
#     u_bat = list(set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['striker'].unique()) | set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['non_striker'].unique()))

#     for df in df_list:

    
#     return random.uniform(0, 1)